# 000_hyp3?

I'm new to hyp3 and InSAR processing in general, so this is exploratory and saved for my own sake. This is with Sentinel-1 burst regular SLCs rather than the OPERA CSLCs found in notebooks/00_SAR_intro.ipynb

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import asf_search
import os

In [ ]:
# read in the SNOTEL sites for a search bbox
gf_snotel = gpd.read_file(
    "https://github.com/egagli/snotel_ccss_stations/raw/main/all_stations.geojson"
    ).set_index("code")
gf_snotel = gf_snotel[gf_snotel.network == "SNOTEL"].reset_index()
sntl_codes = ['589_CO_SNTL', '1185_CO_SNTL', '465_CO_SNTL', 
              '586_CO_SNTL', '629_CO_SNTL', '713_CO_SNTL', '538_CO_SNTL']
gf_snotel_co_4326 = gf_snotel[gf_snotel.code.isin(sntl_codes)]
gf_snotel_co_utm = gf_snotel_co_4326.to_crs("EPSG:32612")

bounds = gf_snotel_co_4326.total_bounds
gf_search = gpd.GeoDataFrame(
    geometry=[box(bounds[0], bounds[1], bounds[2], bounds[3])],
    crs="EPSG:4326"
)
wkt_search = gf_search.geometry.to_wkt().iloc[0]

In [3]:
parms = {
        #'dataset':asf_search.DATASET.OPERA_S1, 
        'dataset':'SLC-BURST',
        #'processingLevel':asf_search.PRODUCT_TYPE.SLC,
        'intersectsWith':wkt_search,
        #'operaBurstID':['T049_103322_IW2', 'T129_275785_IW1'],
        'fullBurstID':['049_103322_IW2', '129_275785_IW1'],
        'start':'2018-01-01T00:00:00Z',
        'end':'2018-01-31T00:00:00Z',
        'flightDirection':asf_search.FLIGHT_DIRECTION.DESCENDING,
        'polarization':asf_search.POLARIZATION.VV,
        #'polarization':'VV+VH',
        'maxResults':100,
}

In [4]:
burst_hits = asf_search.search(**parms)

In [ ]:
from hyp3_sdk import HyP3
import hyp3_sdk as sdk
hyp3 = HyP3(username=os.environ.get('EARTHDATA_USERNAME'), password=os.environ.get('EARTHDATA_PASSWORD'))

In [ ]:
# https://nbviewer.org/github/ASFHyP3/hyp3-sdk/blob/main/docs/sdk_example.ipynb
def get_nearest_neighbors(granule: str, max_neighbors: int | None = None) -> asf_search.ASFSearchResults:
    granule = asf_search.granule_search(granule)[-1]
    stack = reversed([item for item in granule.stack() if item.properties['temporalBaseline'] < 0])
    return asf_search.ASFSearchResults(stack)[:max_neighbors]

In [10]:
granules = [burst_hits[-1].properties.get('url').split("/")[3]]

In [ ]:
from tqdm.auto import tqdm  # For a nice progress bar: https://github.com/tqdm/tqdm#ipython-jupyter-integration
insar_jobs = sdk.Batch()
for reference in tqdm(granules):
    neighbors = get_nearest_neighbors(reference, max_neighbors=1)
    for secondary in neighbors:
        insar_jobs += hyp3.submit_insar_job(reference, 
                                            secondary.properties['sceneName'],
                                            name='insar-test',
                                            )
print(insar_jobs)

  0%|          | 0/1 [00:00<?, ?it/s]

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [12]:
insar_jobs

Batch([Job.from_dict({'credit_cost': 10, 'job_id': 'd17aa057-5efa-4b31-affe-405919483252', 'status_code': 'PENDING', 'job_type': 'INSAR_GAMMA', 'priority': 9950, 'request_time': '2025-02-18T00:19:07+00:00', 'job_parameters': {'include_look_vectors': False, 'include_los_displacement': False, 'include_displacement_maps': False, 'include_inc_map': False, 'include_dem': False, 'include_wrapped_phase': False, 'apply_water_mask': False, 'looks': '20x4', 'phase_filter_parameter': 0.6, 'granules': ['S1A_IW_SLC__1SDV_20180102T131824_20180102T131851_019976_02206D_BCB8', 'S1A_IW_SLC__1SDV_20250125T131849_20250125T131916_057601_0718CF_2F2C']}, 'name': 'insar-test', 'user_id': 'jeghayes'})])

In [ ]:
# how many credits do i have with earthdata??

----
Below is a struggle to use [hyp3-isce2](https://github.com/ASFHyP3/hyp3-isce2) with Docker

In [13]:
granule_1 = burst_hits[-1].properties.get('fileID')
granule_2 = burst_hits[-2].properties.get('fileID')

In [14]:
# disable SSL verification, otherwise timeout on 'requests' to download sar-mpc.eu
cmd = f"""docker run -it --rm \\
  -v /home/jehayes/sh_final/insar/output/desc:/tmp/output \\
  -e EARTHDATA_USERNAME={os.environ.get('EARTHDATA_USERNAME')} \\
  -e EARTHDATA_PASSWORD={os.environ.get('EARTHDATA_PASSWORD')} \\
  -e PYTHONHTTPSVERIFY=0 \\
  ghcr.io/asfhyp3/hyp3-isce2:latest \\
  ++process insar_tops_burst \\
  --reference {granule_1} \\
  --secondary {granule_2} \\
  --looks 20x4 \\
  --apply-water-mask True
"""
#print(cmd)

In [ ]:
# the command runs without error in ~5 mins, but the outputs are not saved...
# TODO: :'(
!{cmd}